In [ ]:
%matplotlib inline
%load_ext rpy2.ipython

In [ ]:
%%R
library(biom)
library(devtools)
library(phyloseq)
library(dplyr)
library(tidyr)
library(SpiecEasi)

In [ ]:
%%R
workDir = '/home/chantal/Chazy/Bulk/515_806/data/'
figDir = '/home/chantal/Chazy/Bulk/515_806/figs/'

physeqDir = '/home/chantal/Chazy/data/phyloseq/'
physeqBulk = 'bulk-core'


nprocs = 20

In [ ]:
%%R

F = file.path(physeqDir, physeqBulk)

physeq.Bulk = readRDS(F)
physeq.Bulk.m = physeq.Bulk %>% sample_data
physeq.Bulk = prune_samples((physeq.Bulk.m$Land_Management != "PAS"),
                           physeq.Bulk)
physeq.Bulk = physeq.Bulk %>% filter_taxa(function(x) sum(x) > 0, TRUE) 
physeq.Bulk.Sparsity = filter_taxa(physeq.Bulk, function(x) sum(x > 0) > (0.25 * length(x)), TRUE)

physeq.Bulk.Sparsity

In [ ]:
%%R
physeq.Bulk.r = rarefy_even_depth(physeq.Bulk.Sparsity, sample.size = 9500, rngseed = 23435, replace = TRUE,
                          trimOTUs = TRUE, verbose = TRUE)

In [ ]:
%%R
physeq.Bulk.r

In [ ]:
%%R
physeq.Bulk.m = physeq.Bulk.r %>% sample_data

In [ ]:
%%R

m = spiec.easi(physeq.Bulk.r, method='mb', lambda.min.ratio=1e-2, 
                            nlambda=15, icov.select.params=list(rep.num=50))




In [ ]:
%%R
str(m)

In [ ]:
%%R
save(m, file = 'data/co-occurrence')

In [ ]:
%%R
#se.m = spiec.easi(mdf_long2, method='glasso', lambda.min.ratio=1e-2,
 #                           nlambda=20, icov.select.params=list(rep.num=50))

In [ ]:
%%R 
#se.gl.amgut <- spiec.easi(amgut1.filt, method='glasso', lambda.min.ratio=1e-2,
#                            nlambda=20, icov.select.params=list(rep.num=50))
#sparcc.amgut <- sparcc(amgut1.filt)
## Define arbitrary threshold for SparCC correlation matrix for the graph
#sparcc.graph <- abs(sparcc.amgut$Cor) >= 0.3
## Create igraph objects
ig.mb <- graph.adjacency(m$refit, mode='undirected')
#ig.gl <- graph.adjacency(se.gl.amgut$refit, mode='undirected')
#ig.sparcc <- graph.adjacency(sparcc.graph, mode='undirected', diag=FALSE)

## set size of vertex proportional to clr-mean
vsize <- 2
am.coord <- layout.fruchterman.reingold(ig.mb)

par(mfrow=c(1,3))
plot(ig.mb, layout=am.coord, vertex.size=vsize, vertex.label=NA, main="MB")
#plot(ig.gl, layout=am.coord, vertex.size=vsize, vertex.label=NA, main="glasso")
#plot(ig.sparcc, layout=am.coord, vertex.size=vsize, vertex.label=NA, main="sparcc")

In [ ]:
%%R
str(m)

In [ ]:
%%R -w 800 -h 800
plot(ig.mb, layout=am.coord, vertex.size=vsize, vertex.label=NA, main="MB")
V(ig.mb)$color <- "white"
